In [ ]:
from pathlib import Path
import os

DAY = "20251201"
Version = "v1"

PROJECT_ROOT = Path(os.getenv("LLMSC_ROOT", ".")).resolve()

DATA_DIR = Path(os.getenv("LLMSC_DATA_DIR", PROJECT_ROOT / "input")).resolve()
OUT_DIR  = Path(os.getenv("LLMSC_OUT_DIR",  PROJECT_ROOT / "runs" / f"{DAY}.{Version}")).resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
RANDOM_SEED = 42
import random, os
import numpy as np
random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"🔒 Random seed set to {RANDOM_SEED} for reproducibility.")

🔒 Random seed set to 42 for reproducibility.


In [ ]:
import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")
USE_LLM = bool(GEMINI_API_KEY)

if not USE_LLM:
    print("⚠️ GEMINI_API_KEY not set → LLM inference cells will be skipped.")

In [ ]:
import google.generativeai as genai
import scanpy as sc
import numpy as np
import pandas as pd
import scipy
import os
import gc
import time

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ==========================================
# 1. API
# ==========================================

genai.configure(api_key=GEMINI_API_KEY)
print("--- Available Models for your Key ---")
try:
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)
except Exception as e:
    print(f"Error: {e}")

--- Available Models for your Key ---
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-

In [ ]:
MODEL_NAME = 'models/gemini-2.5-pro'

# MODEL_NAME = 'models/gemini-3-pro-preview'
# MODEL_NAME = 'models/gemini-2.5-flash'
# MODEL_NAME = 'models/gemini-2.0-flash'

print(f"Using Model: {MODEL_NAME}")
model = genai.GenerativeModel(MODEL_NAME)

Using Model: models/gemini-2.5-pro


In [ ]:
# ==========================================
# 2. Data Loading & Stratified Subsampling
# ==========================================
print("🚀 Building Pan-Cancer T-cell Atlas (ESCA + RC + PACA)...")

# 1. List of filenames for the dataset to be used
datasets = {
    "ESCA": "GSE156728_ESCA_10X.CD4.counts.txt.gz",
    "PACA": "GSE156728_PACA_10X.CD4.counts.txt.gz",
    "RC":   "GSE156728_RC_10X.CD4.counts.txt.gz",
}


# 2. Metadata loading
meta_path = DATA_DIR / "GSE156728_metadata.txt.gz"
print(f"Loading Metadata from {meta_path}...")
meta_df = pd.read_csv(meta_path, sep="\t", index_col=0)

# 3. Settings for stratified subsampling
N_PER_CLUSTER = 100

# 4. Processing
adatas_list = []

for cancer_type, filename in datasets.items():
    counts_path = DATA_DIR / filename
    print(f"\nProcessing {cancer_type} ({filename})...")

    try:
        counts = pd.read_csv(counts_path, sep="\t", index_col=0)
        adata_temp = sc.AnnData(counts.T, dtype='float32')
        del counts
        gc.collect()

        common_cells = adata_temp.obs_names.intersection(meta_df.index)
        adata_temp = adata_temp[common_cells]
        adata_temp.obs = meta_df.loc[common_cells]
        adata_temp.obs['Cancer_Type'] = cancer_type

        sampled_indices = []
        clusters = adata_temp.obs['meta.cluster'].unique()

        for cl in clusters:
            cells_in_cluster = adata_temp.obs[adata_temp.obs['meta.cluster'] == cl].index

            if len(cells_in_cluster) > N_PER_CLUSTER:
                sampled = np.random.choice(cells_in_cluster, N_PER_CLUSTER, replace=False)
            else:
                sampled = cells_in_cluster

            sampled_indices.extend(sampled)

        adata_sub = adata_temp[sampled_indices].copy()

        print(f"  -> Original: {adata_temp.n_obs} cells")
        print(f"  -> Stratified: {adata_sub.n_obs} cells (Balanced)")

        adatas_list.append(adata_sub)

        del adata_temp
        gc.collect()

    except Exception as e:
        print(f"  ⚠️ Error loading {cancer_type}: {e}")

# 5. Concatenate
if len(adatas_list) > 0:
    print("\nMerging datasets...")
    adata = sc.concat(adatas_list, join='outer', fill_value=0)
    adata.obs_names_make_unique()

    print(f"✅ Pan-Cancer Atlas Built: {adata.n_obs} cells, {adata.n_vars} genes")
    print(f"   Cancer Types: {adata.obs['Cancer_Type'].unique()}")
else:
    raise ValueError("No data was loaded. Please verify the path.")

# 6. Preprocessing (Normalization & Log Transformation)
print("\nPreprocessing: Normalizing and Log-transforming...")

adata.layers["counts"] = adata.X.copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers["logcounts"] = adata.X.copy()

adata.obs['Sample_ID'] = adata.obs.index
print("✅ Data Preprocessing Complete.")

save_path = OUT_DIR / "cd4_benchmark_data.h5ad"
print(f"\n💾 Saving Benchmark Data to: {save_path}")
adata.write_h5ad(save_path)

meta_save_path = OUT_DIR / "cd4_benchmark_metadata.csv"
adata.obs.to_csv(meta_save_path)
print(f"📄 Metadata saved to: {meta_save_path}")

print("\nCell counts per cluster (Should be balanced approx ~300 if 3 datasets merged):")
print(adata.obs['meta.cluster'].value_counts().head(10))

🚀 Building Pan-Cancer T-cell Atlas (ESCA + RC + PACA)...
Loading Metadata from /input/GSE156728_metadata.txt.gz...

Processing ESCA (GSE156728_ESCA_10X.CD4.counts.txt.gz)...
  -> Original: 12358 cells
  -> Stratified: 2246 cells (Balanced)

Processing PACA (GSE156728_PACA_10X.CD4.counts.txt.gz)...
  -> Original: 3903 cells
  -> Stratified: 1708 cells (Balanced)

Processing RC (GSE156728_RC_10X.CD4.counts.txt.gz)...
  -> Original: 10105 cells
  -> Stratified: 2051 cells (Balanced)

Merging datasets...
✅ Pan-Cancer Atlas Built: 6005 cells, 24148 genes
   Cancer Types: ['ESCA' 'PACA' 'RC']

Preprocessing: Normalizing and Log-transforming...
✅ Data Preprocessing Complete.

💾 Saving Benchmark Data to: /runs/20251201.v1/cd4_benchmark_data.h5ad
📄 Metadata saved to: /runs/20251201.v1/cd4_benchmark_metadata.csv

Cell counts per cluster (Should be balanced approx ~300 if 3 datasets merged):
meta.cluster
CD4.c01.Tn.TCF7          300
CD4.c06.Tm.ANXA1         300
CD4.c07.Tm.ANXA2         300
CD4.c1